In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import io
import requests
import os

from scipy.stats import zscore
from sklearn import preprocessing, metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv("../../../Datasets/Flows/train.csv")

# Display the DataFrame
display(df)

,service,traffic,total_bytes,total_pkts,pkt_difference,byte_difference,total_data_pkts,payload_ratio,total_payload_volume,fwd_bwd_pkts_diff,...,history_originator,history_responder,proto_1,proto_2,proto_3,pkts_unidirectional_traffic_0,pkts_unidirectional_traffic_1,iat_is_unidirectional_False,iat_is_unidirectional_True,is_attack
0,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
1,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
2,0,netscan,-0.004437,2,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,1,1,1,0,0,1,0,0,1,1
3,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
4,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229452,0,netscan,-0.004437,2,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,1,1,1,0,0,1,0,0,1,1
2229453,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
2229454,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
2229455,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1


In [3]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0     528698
1    1700759
Name: is_attack, dtype: int64

In [4]:
df.groupby('traffic')['traffic'].count()

traffic
apachekiller       1751
arpspoofing        8437
camoverflow     1229862
mqttmalaria        3713
netscan          350214
normal           528698
rudeadyet         54247
slowloris         47519
slowread           5016
Name: traffic, dtype: int64

In [5]:
# Read the CSV file into a DataFrame
df = pd.read_csv("../../../Datasets/Flows/test.csv")

# Display the DataFrame
display(df)

,service,traffic,total_bytes,total_pkts,pkt_difference,byte_difference,total_data_pkts,payload_ratio,total_payload_volume,fwd_bwd_pkts_diff,...,history_originator,history_responder,proto_1,proto_2,proto_3,pkts_unidirectional_traffic_0,pkts_unidirectional_traffic_1,iat_is_unidirectional_False,iat_is_unidirectional_True,is_attack
0,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
1,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
2,0,netscan,-0.004437,2,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,1,1,1,0,0,1,0,0,1,1
3,0,netscan,-0.004437,2,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,1,1,1,0,0,1,0,0,1,1
4,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743148,0,normal,-0.004437,10,-0.122899,-0.003766,5,0.972797,0.954078,0.000492,...,1,1,1,0,0,0,1,0,1,0
743149,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
743150,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1
743151,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1


In [6]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0    176233
1    566920
Name: is_attack, dtype: int64

In [7]:
df.groupby('traffic')['traffic'].count()

traffic
apachekiller       583
arpspoofing       2785
camoverflow     410135
mqttmalaria       1248
netscan         116725
normal          176233
rudeadyet        17743
slowloris        15977
slowread          1724
Name: traffic, dtype: int64